# 概要

動的なニューラルネットワークモデルを作成する。<br>
これまで扱ったSequential APIや関数型APIは宣言的であり、静的な作成方法する方法だった。これら静的なモデル作成には多くのメリットがある。
例えばモデルの保存、クローン作成、共有、構造の表示、分析が簡単にできる。<br>
フレームワーク側で形の推測や型チェックを行え、訓練前にエラーを見つけることが出来る。
また、グラフ全体が静的なグラフなのでデバッグもしやすい。

一方、静的なモデル作成ではループ、形の変更、条件分岐などの動的な動作が必要な場合は対応できない。<br>
いま挙げたような命令型プログラミングのスタイルを扱うにはサブクラスAPIを使用する。

# サブクラスAPI

Modelクラスのサブクラスを定義する。<br>
コンストラクタの中に必要な層を定義し、callメソッドに各層での計算を実装する。

今回は関数型APIで実装した出力を複数持つワイド・アンド・ディープニューラルネットワークを実装する。

## パッケージインポート

In [40]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

print(f'tensorflow ver.{tf.__version__}')
print(f'keras ver.{keras.__version__}')

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

tensorflow ver.2.5.0
keras ver.2.5.0


## データロードと前処理

###  データロード（sklearn.datasets.fetch_california_housing）

In [41]:
housing = fetch_california_housing()

x_train_full, x_test, y_train_full, y_test = train_test_split(housing.data, housing.target)

### validation分割

In [42]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train_full, y_train_full)

# データサイズを確認
print(f'x_train.shape : {x_train.shape}')
print(f'y_train.shape : {y_train.shape}')
print(f'x_valid.shape : {x_valid.shape}')
print(f'y_valid.shape : {y_valid.shape}')

x_train.shape : (11610, 8)
y_train.shape : (11610,)
x_valid.shape : (3870, 8)
y_valid.shape : (3870,)


trainデータを確認

In [43]:
pd_x_train = pd.DataFrame(x_train, columns=housing.feature_names)
display(pd_x_train)
pd_x_train.info()
pd_x_train.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,7.1035,52.0,7.269565,1.043478,832.0,2.411594,38.55,-121.48
1,3.1125,11.0,5.705696,1.071994,3264.0,2.582278,39.75,-121.80
2,5.6876,17.0,6.348266,1.024566,2292.0,3.312139,37.27,-121.86
3,8.2673,21.0,7.539615,0.957173,1396.0,2.989293,37.87,-122.03
4,3.2589,42.0,4.215116,1.063953,493.0,2.866279,33.88,-118.34
...,...,...,...,...,...,...,...,...
11605,4.1411,30.0,4.978998,0.972536,2008.0,3.243942,34.11,-117.88
11606,4.3958,10.0,6.154506,1.013948,3528.0,3.785408,34.09,-117.39
11607,3.5192,24.0,5.909707,1.094808,726.0,1.638826,33.01,-117.06
11608,5.3226,18.0,5.567742,0.890323,1186.0,3.825806,37.60,-122.06


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11610 entries, 0 to 11609
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   MedInc      11610 non-null  float64
 1   HouseAge    11610 non-null  float64
 2   AveRooms    11610 non-null  float64
 3   AveBedrms   11610 non-null  float64
 4   Population  11610 non-null  float64
 5   AveOccup    11610 non-null  float64
 6   Latitude    11610 non-null  float64
 7   Longitude   11610 non-null  float64
dtypes: float64(8)
memory usage: 725.8 KB


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,11610.000000,11610.000000,11610.000000,11610.000000,11610.000000,11610.000000,11610.000000,11610.000000
mean,3.863226,28.641947,5.425370,1.094283,1421.994057,3.097217,35.641338,-119.578549
std,1.892678,12.588064,2.146476,0.388205,1105.357225,12.469923,2.137063,1.997922
min,0.499900,1.000000,0.888889,0.375000,5.000000,0.692308,32.540000,-124.270000
25%,2.562500,18.000000,4.447368,1.005885,782.000000,2.429296,33.940000,-121.790000
50%,3.531300,29.000000,5.232284,1.048327,1162.500000,2.819278,34.270000,-118.520000
75%,4.738100,37.000000,6.050518,1.098121,1727.000000,3.281785,37.720000,-118.020000
max,15.000100,52.000000,59.875000,15.312500,15507.000000,1243.333333,41.920000,-114.310000


### 前処理

#### スケーリング

### [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)
データの標準化を行う。

代表的なメソッドは以下：

|メソッド|説明|
|---|---|
|fit()|標準化するための平均と分散を計算する。|
|trasform()|（事前に計算した平均と分散を使用して）標準化を行う。|
|fit_transform()|平均と分散を計算し、標準化を行う。|

In [44]:
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_valid = scaler.transform(x_valid)  # x_trainの平均・分散を使用する（のはなぜ？）
x_test = scaler.transform(x_test)  # x_trainの平均・分散を使用する（のはなぜ？）

#### inputデータ分割

In [45]:
# ワイドパス、ディープパス用で分割
x_train_A, x_train_B = x_train[:, :5], x_train[:, 2:]
x_valid_A, x_valid_B = x_valid[:, :5], x_valid[:, 2:]
x_test_A, x_test_B = x_test[:, :5], x_test[:, 2:]

## モデル作成

### レイヤ構成を定義

In [46]:
class WideAndDeepModel(keras.models.Model):
    def __init__(self, units=30, activation='relu', **kwargs):
        # super().__init__(**kwargs)  # handles standard args (ex : name)
        super().__init__(**kwargs)  # 標準引数(ex : name)を処理する←これ2回いる？
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)  # keras.Model.output属性があるので競合しないように
        self.aux_output = keras.layers.Dense(1)

    def call(self, inputs):
        input_A, input_B = inputs
        hidden_1 = self.hidden1(input_B)
        hidden_2 = self.hidden2(hidden_1)
        concat = keras.layers.concatenate([input_A, hidden_2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden_2)
        return main_output, aux_output

model = WideAndDeepModel(units=30, activation='relu')


この例では関数型APIとよく似ているが、input層を作る必要がないところが異なる。<br>
またコンストラクタ内で層を作成しcallメソッドの中で層の利用の仕方を実装するため、層の作成と利用を分離出来る。<br>
これにより関数型APIよりも柔軟にモデル作成が出来る。

## モデルのコンパイル

柔軟さの代償もある。<br>
モデルのアーキテクチャがcallメソッドに隠されてしまうため、Kerasが簡単にモデルを精査できなくなる。<br>
また、保存やクローン作成もできなくなる。<br>
summaryメソッドを呼び出しても層のリストは得られるが接続情報は得られない。

In [47]:
model.compile(loss="mse", loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD())

# model.summary()  # errorになる

# 学習と評価

### コールバック設定

In [48]:
# 早期打ち切りのコールバック関数
# 学習打ち切り時に性能が最高だった時の重みを復元するので最良モデルの保存と復元は不要
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)  # patienceで指定したエポック数学習が進まなかったときに学習を打ち切る

In [49]:
class PrintValTrainRatioCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print("\nval/train: {:.2f}".format(logs["val_loss"] / logs["loss"]))

print_valid_train_ration_cb = PrintValTrainRatioCallback()

In [50]:
import os

# ログ出力のルートディレクトリ
root_dir = os.path.join(os.curdir, 'my_logs')

# ログディレクトリ名を生成する関数
def get_run_logdir():
    import time
    run_id = time.strftime('run_%Y_%m_%d-%H_%M_%S')
    return os.path.join(root_dir, run_id)

run_logdir = get_run_logdir()

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

## 学習

学習時もそれぞれの出力に対してラベルを指定する必要がある。<br>
今回の例では両方同じものを予測しているので同じラベルを渡す。

In [51]:
history = model.fit([x_train_A, x_train_B], [y_train, y_train], epochs=10, validation_data=([x_valid_A, x_valid_B], [y_valid, y_valid]), callbacks=[tensorboard_cb, early_stopping_cb, print_valid_train_ration_cb])

Epoch 1/10
363/363 [==============================] - 4s 5ms/step - loss: 0.9263 - output_1_loss: 0.8003 - output_2_loss: 2.0607 - val_loss: 0.6241 - val_output_1_loss: 0.5434 - val_output_2_loss: 1.3507

val/train: 0.67
Epoch 2/10
363/363 [==============================] - 1s 3ms/step - loss: 0.5869 - output_1_loss: 0.5167 - output_2_loss: 1.2183 - val_loss: 0.5623 - val_output_1_loss: 0.4975 - val_output_2_loss: 1.1461

val/train: 0.96
Epoch 3/10
363/363 [==============================] - 1s 2ms/step - loss: 0.5912 - output_1_loss: 0.5401 - output_2_loss: 1.0504 - val_loss: 0.5487 - val_output_1_loss: 0.4961 - val_output_2_loss: 1.0217

val/train: 0.93
Epoch 4/10
363/363 [==============================] - 1s 2ms/step - loss: 0.5123 - output_1_loss: 0.4688 - output_2_loss: 0.9041 - val_loss: 0.5100 - val_output_1_loss: 0.4689 - val_output_2_loss: 0.8793

val/train: 1.00
Epoch 5/10
363/363 [==============================] - 1s 2ms/step - loss: 0.4813 - output_1_loss: 0.4466 - output_2_

## 学習結果の可視化

In [52]:
%load_ext tensorboard
%tensorboard --logdir ./my_logs

Reusing TensorBoard on port 6006 (pid 29808), started 1:29:14 ago. (Use '!kill 29808' to kill it.)

In [53]:
# import pandas as pd
# import matplotlib.pyplot as plt
# pd.DataFrame(history.history).plot(figsize=(8, 5))  # historyはエポック毎のloss, val_lossを保持する
# plt.grid(True)
# plt.gca().set_ylim(0, 1) # 縦の範囲を 0 から 1 までに
# plt.show()

## 学習結果の評価

In [54]:
total_loss, main_loss, aux_loss = model.evaluate((x_test_A, x_test_B), (y_test, y_test))
total_loss, main_loss, aux_loss

162/162 [==============================] - 0s 1ms/step - loss: 0.4531 - output_1_loss: 0.4338 - output_2_loss: 0.6266


(0.45308640599250793, 0.4338088035583496, 0.6265859007835388)

# 学習済みモデルを使った予測

In [55]:
# サンプル用にデータサイズを限定
x_new_A, x_new_B = x_test_A[:3], x_test_B[:3]

y_pred_main, y_pred_aux = model.predict((x_new_A, x_new_B))
print(f'predict_main : {y_pred_main.reshape(-1)}')
print(f'predict_aux : {y_pred_aux.reshape(-1)}')
print(f'correct : {y_test[:3]}')

predict_main : [2.523777  2.3229446 1.4488592]
predict_aux : [2.4256957 2.6327195 1.4161401]
correct : [2.674 1.711 1.252]
